In [13]:
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [14]:
tfd = tfp.distributions

In [18]:
# Confirm GPU in use
tf.config.list_physical_devices('GPU')

[]

In [19]:
# Parameters
P = 10
N = 10 ** 2
SEED = 1729

In [20]:
# Ground truth
np.random.seed(SEED)
true_transmission_rate = np.random.beta(2, 10, P)
true_occurrence_rate = np.random.beta(2, 10, P)
base_rate = np.random.beta(2, 10, 1)

In [21]:
# Simulate data
data = {}
for p in range(P):
    occurrence = np.random.binomial(1, true_occurrence_rate[p], N)
    transmission = occurrence * np.random.binomial(1, true_transmission_rate[p], N)
    data[f'O{p+1}'] = occurrence
    data[f'T{p+1}'] = transmission
data['T0'] = np.random.binomial(1, base_rate, N)
X = pd.DataFrame(data)

z = X.loc[:, X.columns.str.startswith('T')].sum(axis=1)
X = X.loc[:, X.columns.str.startswith('O')]
y = (z > 0).astype(int)

In [22]:
# Convert to tensors
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)
# Move to GPU
X = X + tf.fill(X.shape, 0.0)
y = y + tf.fill(y.shape, 0.0)

In [23]:
# Define log-likelihood
@tf.function
def censored_poisbinom_loglike(theta, rho):
    if tf.math.reduce_any(tf.math.logical_or(theta <= 0., theta >= 1.)):
        return -np.inf
    if tf.math.logical_or(rho <= 0., rho >= 1.):
        return -np.inf
    log1m_theta = tf.math.log(1-theta)
    s = tf.einsum('ij,j->i', X, log1m_theta) + tf.math.log(1-rho)
    ll = tf.math.reduce_sum(tf.where(y == 1, tfp.math.log1mexp(s), s))
    return ll

In [24]:
# Define negative log-likelihood and use AD to compute gradients
@tf.function
def censored_poisbinom_negloglike(params):
    theta, rho = tf.split(params, [P, 1], axis=0)
    # need to take these back down to vectors and scalars:
    theta = tf.reshape(theta,(P,))
    rho = tf.reshape(rho,())
    return -1 * censored_poisbinom_loglike(theta, rho)

@tf.function
def censored_poisbinom_negloglike_and_grad(params):
    return tfp.math.value_and_gradient(
        censored_poisbinom_negloglike, 
        params
    )

In [25]:
# Approximate MLE using gradient descent
start = tf.fill(P + 1, 0.5)

optim_results = tfp.optimizer.bfgs_minimize(
    censored_poisbinom_negloglike_and_grad, start, tolerance=1e-8
)

est_params = optim_results.position.numpy()
est_serr = np.sqrt(np.diagonal(optim_results.inverse_hessian_estimate.numpy()))
display(pd.DataFrame(
    np.c_[est_params, est_serr, np.concatenate([true_transmission_rate, base_rate])],
    columns=['estimate', 'std err', 'true_val'],
    index=[f'theta_{i}' for i in range(1, P + 1)] + ['rho']
))

,estimate,std err,true_val
theta_1,5.293956e-23,4.730305,0.112290
theta_2,2.201886e-01,0.935157,0.215440
theta_3,9.140683e-01,35.844563,0.134004
theta_4,1.091509e-01,4.519799,0.034960
theta_5,1.114019e-01,4.377872,0.140240
theta_6,4.558466e-01,0.943219,0.389962
theta_7,5.473991e-02,8.253405,0.061519
theta_8,1.396171e-01,8.689433,0.096669
theta_9,4.138582e-01,0.993721,0.086170
theta_10,1.258933e-01,17.242455,0.052647


In [26]:
# Set model parameters
nuts_samples = 1000
nuts_burnin = 200
init_step_size=.3
init = [est_params[:P], est_params[-1]]

In [27]:
# Fit model
@tf.function
def nuts_sampler(init):
    nuts_kernel = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=censored_poisbinom_loglike, 
        step_size=init_step_size,
    )
    adapt_nuts_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
        inner_kernel=nuts_kernel,
        num_adaptation_steps=nuts_burnin,
        step_size_getter_fn=lambda pkr: pkr.step_size,
        log_accept_prob_getter_fn=lambda pkr: pkr.log_accept_ratio,
        step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(step_size=new_step_size)
    )

    samples_nuts_, stats_nuts_ = tfp.mcmc.sample_chain(
        num_results=nuts_samples,
        current_state=init,
        kernel=adapt_nuts_kernel,
        num_burnin_steps=nuts_burnin,
        parallel_iterations=10,
        trace_fn=None
    )
    return samples_nuts_, stats_nuts_

start = time.time()
samples_nuts, stats_nuts = nuts_sampler(init)
print(f"{time.time() - start:.02f} seconds elapsed")

4.83 seconds elapsed


In [28]:
# View results
trace_rho = stats_nuts.numpy()
trace_theta = samples_nuts.numpy()
est_nuts = np.r_[trace_theta.mean(axis=0), trace_rho.mean()]
std_nuts = np.r_[trace_theta.std(axis=0), trace_rho.std()]
# assemble and print
display(pd.DataFrame(
    np.c_[est_nuts, std_nuts, np.concatenate([true_transmission_rate, base_rate])],
    columns=['estimate', 'std err', 'true_val'],
    index=[f'theta_{i}' for i in range(1, P + 1)] + ['rho']
))

,estimate,std err,true_val
theta_1,0.134276,0.158168,0.112290
theta_2,0.147452,0.128316,0.215440
theta_3,0.748803,0.140446,0.134004
theta_4,0.085003,0.055728,0.034960
theta_5,0.096893,0.069826,0.140240
theta_6,0.396143,0.146952,0.389962
theta_7,0.123677,0.085235,0.061519
theta_8,0.170643,0.169399,0.096669
theta_9,0.316419,0.197264,0.086170
theta_10,0.224810,0.112296,0.052647
